# Image Classification using Convolutional Neural Networks (CNNs)
- In this project, I classified the images in CIFAR10 dataset into 10 categories (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck) using Convolutional Neural Networks(CNNs).  

- To this end, I implemented necessary network components (e.g. residual blocks) using nn. Module class and completed whole CNNs with those blocks. Then, I experimented those network architectures using train/testing pipeline.

In [ ]:
# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your google drive.
import os
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# Specify the directory path where `assignemnt1.ipynb` exists.
# For example, if you saved `assignment1.ipynb` in `/gdrive/My Drive/CS471/assignment2` directory,
# then set root = '/gdrive/My Drive/CS471/assignment2'
root = '/gdrive/My Drive/CS71/assignment2'

In [ ]:
from PIL import Image
from tqdm import tqdm
from pathlib import Path
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

### MLP Block [(Illustration)](https://docs.google.com/drawings/d/17P4EZfF8ZoU6lllhg3quGHgDY8ol7wPEWOs3XutpHoU/edit?usp=sharing)  

In [ ]:
class MLPBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MLPBlock, self).__init__()
        """
        Initialize a basic multi-layer perceptron module components.
        Illustration: https://docs.google.com/drawings/d/17P4EZfF8ZoU6lllhg3quGHgDY8ol7wPEWOs3XutpHoU/edit?usp=sharing

        Instructions:
            1. Implement an algorithm that initializes necessary components as illustrated in the above link.
            2. Initialized network components will be referred in `forward` method
               for constructing the dynamic computational graph.

        Args:
            1. in_channels (int): Number of channels in input.
            2. out_channels (int): Number of channels to be produced.
        """
        self.fc1 = nn.Linear(in_channels, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512,128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, out_channels)
        self.bn3 = nn.BatchNorm1d(out_channels)
        self.act = nn.ReLU()

    def forward(self, x):
        """
        Feed-forward data 'x' through the module.

        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link
               using the initialized components in __init__ method.

        Args:
            1. x (torch.FloatTensor): A tensor of shape (B, in_channels)
            .
        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, out_channels).
        """
        output = self.act(self.bn1(self.fc1(x)))
        output = self.act(self.bn2(self.fc2(output)))
        output = self.act(self.bn3(self.fc3(output)))
        return output

### Convolutional Block[(Illustration)](https://docs.google.com/drawings/d/1ZrJAfY0GwfQ1IcmFuJaroFF5rj7FQZ2nZ3kjQXywhNs/edit?usp=sharing)

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1,
                 padding=1):
        super(ConvBlock, self).__init__()
        """
        Initialize a basic convolutional layer module components.
        Illustration: https://docs.google.com/drawings/d/1ZrJAfY0GwfQ1IcmFuJaroFF5rj7FQZ2nZ3kjQXywhNs/edit?usp=sharing

        Args:
            1. in_channels (int): Number of channels in the input.
            2. out_channels (int): Number of channels produced.
            3. kernel_size (int) : Size of the kernel used in conv layer (Default:3)
            4. stride (int) : Stride of the convolution (Default:1)
            5. padding (int) : Zero-padding added to both sides of the input (Default:1)
        """
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Feed-forward the data 'x' through the module.
        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link
               using the initialized components in __init__ method.

        Args:
            1. x (torch.FloatTensor): A tensor of shape (B, in_channels, H, W).

        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, out_channels, H, W).
        """
        convo_lay = self.conv(x)
        norm_lay = self.bn(convo_lay)
        output = self.relu(norm_lay)
        return output

### ResBlockPlain [(Illustration)](https://docs.google.com/drawings/d/1bpWUIZ8uwGmfhu-tKAa01l1qBl5oouxCBZQTy5pkRaQ/edit?usp=sharing) (10pt)

In [ ]:
class ResBlockPlain(nn.Module):
    def __init__(self, in_channels):
        super(ResBlockPlain, self).__init__()
        """Initialize a residual block module components.

        Illustration: https://docs.google.com/drawings/d/1bpWUIZ8uwGmfhu-tKAa01l1qBl5oouxCBZQTy5pkRaQ/edit?usp=sharing

        Instructions:
            1. Implement an algorithm that initializes necessary components as illustrated in the above link.
            2. Initialized network components will be referred in `forward` method
               for constructing the dynamic computational graph.

        Args:
            1. in_channels (int): Number of channels in the input.
        """
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        """Feed-forward the data `x` through the network.

        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link
               using the initialized components in __init__ method.

        Args:
            1. x (torch.FloatTensor): An tensor of shape (B, in_channels, H, W).

        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, in_channels, H, W).
        """
        temp = x
        convo_lay1 = self.conv1(x)
        norm_lay1 = self.bn1(convo_lay1)
        output1 = self.relu(norm_lay1)
        convo_lay2 = self.conv2(output1)
        norm_lay2 = self.bn2(convo_lay2)
        norm_lay2 += temp
        output = self.relu(norm_lay2)
        return output

### ResBlockBottleneck [(Illustration)](https://docs.google.com/drawings/d/1t55ibttP-X-8vPWYWFangN9pYdyDD6pVO7IJ_9tPgSA/edit?usp=sharing) 

In [ ]:
class ResBlockBottleneck(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(ResBlockBottleneck, self).__init__()
        """Initialize a residual block module components.

        Illustration: https://docs.google.com/drawings/d/1t55ibttP-X-8vPWYWFangN9pYdyDD6pVO7IJ_9tPgSA/edit?usp=sharing

        Instructions:
            1. Implement an algorithm that initializes necessary components as illustrated in the above link.
            2. Initialized network components will be referred in `forward` method
               for constructing the dynamic computational graph.

        Args:
            1. in_channels (int): Number of channels in the input.
            2. hidden_channels (int): Number of hidden channels produced by the first ConvLayer module.
        """
        self.conv1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(hidden_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(hidden_channels)
        self.conv3 = nn.Conv2d(hidden_channels, in_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        """Feed-forward the data `x` through the network.

        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link
               using the initialized components in __init__ method.

        Args:
            1. x (torch.FloatTensor): An tensor of shape (B, in_channels, H, W).

        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, in_channels, H, W).
        """
        temp = x
        convo_lay1 = self.conv1(x)
        norm_lay1 = self.bn1(convo_lay1)
        output1 = self.relu(norm_lay1)
        convo_lay2 = self.conv2(output1)
        norm_lay2 = self.bn2(convo_lay2)
        output2 = self.relu(norm_lay2)
        convo_lay3 = self.conv3(output2)
        norm_lay3 = self.bn3(convo_lay3)
        norm_lay3 += temp
        output = self.relu(norm_lay3)
        return output

### InceptionBlock[(Illustration)](https://docs.google.com/drawings/d/1ph_2qLcAWm_voJKWWAB-_Id0PtpJ73W4GEQQ33m3WpY/edit?usp=sharing)

In [ ]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionBlock, self).__init__()
        """Initialize a basic InpcetionBlock module components.

        Illustration: https://docs.google.com/drawings/d/1ph_2qLcAWm_voJKWWAB-_Id0PtpJ73W4GEQQ33m3WpY/edit?usp=sharing
        Instructions:
            1. Implement an algorithm that initializes necessary components as illustrated in the above link.
            2. Initialized network components will be referred in `forward` method
               for constructing the dynamic computational graph.

        Args:
            1. in_channels (int): Number of channels in the input.
            2. out_channels (int): Number of channels in the final output.
        """
        assert out_channels%8==0, 'out channel should be mutiplier of 8'

        # Branch 1
        self.branch1_conv1 = nn.Conv2d(in_channels, int(out_channels/4), kernel_size=1, stride=1, padding=0, bias=False)
        self.branch1_bn1 = nn.BatchNorm2d(int(out_channels/4))
        self.branch1_relu = nn.ReLU(inplace=True)

        # Branch 2
        self.branch2_conv1 = nn.Conv2d(in_channels, int(out_channels/2), kernel_size=1, stride=1, padding=0, bias=False)
        self.branch2_bn1 = nn.BatchNorm2d(int(out_channels/2))
        self.branch2_relu = nn.ReLU(inplace=True)
        self.branch2_conv2 = nn.Conv2d(int(out_channels/2), int(out_channels/2), kernel_size=3, stride=1, padding=1, bias=False)
        self.branch2_bn2 = nn.BatchNorm2d(int(out_channels/2))

        # Branch 3
        self.branch3_conv1 = nn.Conv2d(in_channels, int(out_channels/8), kernel_size=1, stride=1, padding=0, bias=False)
        self.branch3_bn1 = nn.BatchNorm2d(int(out_channels/8))
        self.branch3_relu = nn.ReLU(inplace=True)
        self.branch3_conv2 = nn.Conv2d(int(out_channels/8), int(out_channels/8), kernel_size=5, stride=1, padding=2, bias=False)
        self.branch3_bn2 = nn.BatchNorm2d(int(out_channels/8))

        #Branch 4
        self.branch4_maxpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.branch4_conv1 = nn.Conv2d(in_channels, int(out_channels/8), kernel_size=1, stride=1, padding=0, bias=False)
        self.branch4_bn1 = nn.BatchNorm2d(int(out_channels/8))
        self.branch4_relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """Feed-forward the data `x` through the module.

        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link
               using the initialized components in the __init__ method.

        Args:
            1. x (torch.FloatTensor): A tensor of shape (B, in_channels, H, W).

        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, out_channels, H, W).

        """
        #branch 1
        bra1_convo_lay1 = self.branch1_conv1(x)
        bra1_norm_lay1 = self.branch1_bn1(bra1_convo_lay1)
        bra1_output1 = self.branch1_relu(bra1_norm_lay1)

        #branch 2
        bra2_convo_lay1 = self.branch2_conv1(x)
        bra2_norm_lay1 = self.branch2_bn1(bra2_convo_lay1)
        bra2_output1 = self.branch2_relu(bra2_norm_lay1)
        bra2_convo_lay2 = self.branch2_conv2(bra2_output1)
        bra2_norm_lay2 = self.branch2_bn2(bra2_convo_lay2)
        bra2_output2 = self.branch2_relu(bra2_norm_lay2)

        #branch 3
        bra3_convo_lay1 = self.branch3_conv1(x)
        bra3_norm_lay1 = self.branch3_bn1(bra3_convo_lay1)
        bra3_output1 = self.branch3_relu(bra3_norm_lay1)
        bra3_convo_lay2 = self.branch3_conv2(bra3_output1)
        bra3_norm_lay2 = self.branch3_bn2(bra3_convo_lay2)
        bra3_output2 = self.branch3_relu(bra3_norm_lay2)

        #branch 4
        bra4_max_lay1 = self.branch4_maxpool(x)
        bra4_convo_lay1 = self.branch4_conv1(bra4_max_lay1)
        bra4_norm_lay1 = self.branch4_bn1(bra4_convo_lay1)
        bra4_output1 = self.branch4_relu(bra4_norm_lay1)

        #Concatenation part
        output = torch.cat((bra1_output1, bra2_output2, bra3_output2, bra4_output1), dim=1)
        return output

In [ ]:
class MyNetworkExample(nn.Module):
    def __init__(self, nf, block_type='mlp'):
        super(MyNetworkExample, self).__init__()
        """Initialize an entire network module components.

        Instructions:
            1. Implement an algorithm that initializes necessary components.
            2. Initialized network components will be referred in `forward` method
               for constructing the dynamic computational graph.

        Args:
            1. nf (int): Number of input channels for the first nn.Linear Module. An abbreviation for num_filter.
            2. block_type (str, optional): Type of blocks to use. ('mlp'. default: 'mlp')
        """
        if block_type == 'mlp':
            block = MLPBlock
            # Since shape of input image is 3 x 32 x 32, the size of flattened input is 3*32*32.
            self.mlp = block(3*32*32, nf)
            self.fc = nn.Linear(nf, 10)
        else:
            raise Exception(f"Wrong type of block: {block_type}.Expected : mlp")

    def forward(self, x):
        """Feed-forward the data `x` through the network.

        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link
               using the initialized network components in __init__ method.
        Args:
            1. x (torch.FloatTensor): An image tensor of shape (B, 3, 32, 32).

        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, 10).
        """
        output = self.mlp(x.view(x.size()[0], -1))
        output = self.fc(output)
        return output

### MyNetwork[(Illustration)](https://docs.google.com/drawings/d/1kk_oU9ZtaZlAchKruXykvDuBNpt-wVZ8pyBCIE7z3cU/edit?usp=sharing)

There are two functions implemented in this section.

In [ ]:
class MyNetwork(nn.Module):
    def __init__(self, nf, block_type='conv', num_blocks=[1, 1, 1]):
        super(MyNetwork, self).__init__()
        """Initialize an entire network module components.

        Illustration: https://docs.google.com/drawings/d/1kk_oU9ZtaZlAchKruXykvDuBNpt-wVZ8pyBCIE7z3cU/edit?usp=sharing

        Instructions:
            1. Implement an algorithm that initializes necessary components as illustrated in the above link.
            2. Initialized network components will be referred in `forward` method
               for constructing the dynamic computational graph.

        Args:
            1. nf (int): Number of output channels for the first nn.Conv2d Module. An abbreviation for num_filter.
            2. block_type (str, optional): Type of blocks to use. ('conv' | 'resPlain' | 'resBottleneck' | 'inception'. default: 'conv')
            3. num_blocks (list or tuple, optional): A list or tuple of length 3.
               Each item at i-th index indicates the number of blocks at i-th Layer.
               (default: [1, 1, 1])
        """

        self.block_type = block_type
        # Define blocks according to block_type
        if self.block_type == 'conv':
            block = ConvBlock
            block_args = lambda x: (x, x, 3, 1, 1)
        elif self.block_type == 'resPlain':
            block = ResBlockPlain
            block_args = lambda x: (x,)
        elif self.block_type == 'resBottleneck':
            block = ResBlockBottleneck
            block_args = lambda x: (x, x//2)
        elif self.block_type == 'inception':
            block = InceptionBlock
            block_args = lambda x: (x, x)
        else:
            raise Exception(f"Wrong type of block: {block_type}")

        # Define block layer by stacking multiple blocks.
        # You don't need to modify it. Just use these block layers in forward function.
        self.block1 = nn.Sequential(*[block(*block_args(nf)) for _ in range(num_blocks[0])])
        self.block2 = nn.Sequential(*[block(*block_args(nf*2)) for _ in range(num_blocks[1])])
        self.block3 = nn.Sequential(*[block(*block_args(nf*4)) for _ in range(num_blocks[2])])

        # before block 1
        self.conv1 = nn.Conv2d(3, nf, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(nf)
        self.relu = nn.ReLU(inplace=True)
        self.max_1 = nn.MaxPool2d(kernel_size=2, stride=2)

        #before block 2
        self.conv2 = nn.Conv2d(nf, nf*2, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(nf*2)
        self.max_2 = nn.MaxPool2d(kernel_size=2, stride=2)

        #before block 3
        self.conv3 = nn.Conv2d(nf*2, nf*4, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(nf*4)
        self.max_3 = nn.MaxPool2d(kernel_size=2, stride=2)

        #after block3
        self.avg_1 = nn.AdaptiveAvgPool2d((1,1))
        self.flat = nn.Flatten()
        self.line = nn.Linear(nf*4, 10)

    def forward(self, x):
        """Feed-forward the data `x` through the network.

        Instructions:
            1. Construct the feed-forward computational graph as illustrated in the link
               using the initialized network components in __init__ method.
        Args:
            1. x (torch.FloatTensor): An image tensor of shape (B, 3, 32, 32).

        Returns:
            1. output (torch.FloatTensor): An output tensor of shape (B, 10).
        """

        convo_lay1 = self.conv1(x)
        norm_lay1 = self.bn1(convo_lay1)
        output1 = self.relu(norm_lay1)
        max_pool_1 = self.max_1(output1)
        new_block1 = self.block1(max_pool_1)
        convo_lay2 = self.conv2(new_block1)
        norm_lay2 = self.bn2(convo_lay2)
        output2 = self.relu(norm_lay2)
        max_pool_2 = self.max_2(output2)
        new_block2 = self.block2(max_pool_2)
        convo_lay3 = self.conv3(new_block2)
        norm_lay3 = self.bn3(convo_lay3)
        output3 = self.relu(norm_lay3)
        max_pool_3 = self.max_3(output3)
        new_block3 = self.block3(max_pool_3)
        avg_pool_1 = self.avg_1(new_block3)
        output = self.flat(avg_pool_1)
        output = self.line(output)
        return output

In [ ]:
# Configurations & Hyper-parameters

from easydict import EasyDict as edict

# set manual seeds
torch.manual_seed(470)
torch.cuda.manual_seed(470)

args = edict()

# basic options
args.name = 'main'                   # experiment name.
args.ckpt_dir = 'ckpts'              # checkpoint directory name.
args.ckpt_iter = 1000                # how frequently checkpoints are saved.
args.ckpt_reload = 'best'            # which checkpoint to re-load.
args.gpu = True                      # whether or not to use gpu.

# network options
args.num_filters = 16                # number of output channels in the first nn.Conv2d module in MyNetwork.
args.block_type = 'mlp'              # type of block. ('mlp' | 'conv' | 'resPlain' | 'resBottleneck' | 'inception').
args.num_blocks = [5, 5, 5]          # number of blocks in each Layer.

# data options
args.dataroot = 'dataset/cifar10'    # where CIFAR10 images exist.
args.batch_size = 128                # number of mini-batch size.

# training options
args.lr = 0.1                        # learning rate.
args.epoch = 100                     # training epoch.

# tensorboard options
args.tensorboard = True              # whether or not to use tensorboard logging.
args.log_dir = 'logs'                # to which tensorboard logs will be saved.
args.log_iter = 100                  # how frequently logs are saved.

In [ ]:
# Basic settings
device = 'cuda' if torch.cuda.is_available() and args.gpu else 'cpu'

result_dir = Path(root) / 'results'
result_dir.mkdir(parents=True, exist_ok=True)

global_step = 0
best_accuracy = 0.

In [ ]:
# Define train/test data loaders
# Use data augmentation in training set to mitigate overfitting.
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

train_dataset = CIFAR10(args.dataroot, download=True, train=True, transform=train_transform)
test_dataset = CIFAR10(args.dataroot, download=True, train=False, transform=test_transform)

train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False)

In [ ]:
# Setup tensorboard.
if args.tensorboard:
    from torch.utils.tensorboard import SummaryWriter
    %load_ext tensorboard
    %tensorboard --logdir "/gdrive/My Drive/{str(result_dir).replace('/gdrive/My Drive/', '')}"
else:
    writer = None

In [ ]:
def train_net(net, optimizer, scheduler, block_type, writer):
    global_step = 0
    best_accuracy = 0

    for epoch in range(args.epoch):
        # Here starts the train loop.
        net.train()
        for batch_idx, (x, y) in enumerate(train_dataloader):

            global_step += 1

            #  Send `x` and `y` to either cpu or gpu using `device` variable.
            x = x.to(device=device)
            y = y.to(device=device)

            # Feed `x` into the network, get an output, and keep it in a variable called `logit`.
            logit = net(x)

            # Compute accuracy of this batch using `logit`, and keep it in a variable called 'accuracy'.
            accuracy = (logit.argmax(1) == y).float().mean()

            # Compute loss using `logit` and `y`, and keep it in a variable called `loss`.
            loss = nn.CrossEntropyLoss()(logit, y)

            # flush out the previously computed gradient.
            optimizer.zero_grad()

            # backward the computed loss.
            loss.backward()

            # update the network weights.
            optimizer.step()

            if global_step % args.log_iter == 0 and writer is not None:
                # Log loss and accuracy values using `writer`. Use `global_step` as a timestamp for the log.
                writer.add_scalar('train_loss', loss, global_step)
                writer.add_scalar('train_accuracy', accuracy, global_step)

            if global_step % args.ckpt_iter == 0:
                # Save network weights in the directory specified by `ckpt_dir` directory.
                torch.save(net.state_dict(), f'{ckpt_dir}/{global_step}.pt')

        # Here starts the test loop.
        net.eval()
        with torch.no_grad():
            test_loss = 0.
            test_accuracy = 0.
            test_num_data = 0.
            for batch_idx, (x, y) in enumerate(test_dataloader):
                # Send `x` and `y` to either cpu or gpu using `device` variable..
                x = x.to(device=device)
                y = y.to(device=device)

                # Feed `x` into the network, get an output, and keep it in a variable called `logit`.
                logit = net(x)

                # Compute loss using `logit` and `y`, and keep it in a variable called `loss`.
                loss = nn.CrossEntropyLoss()(logit, y)

                # Compute accuracy of this batch using `logit`, and keep it in a variable called 'accuracy'.
                accuracy = (logit.argmax(dim=1) == y).float().mean()

                test_loss += loss.item()*x.shape[0]
                test_accuracy += accuracy.item()*x.shape[0]
                test_num_data += x.shape[0]

            test_loss /= test_num_data
            test_accuracy /= test_num_data

            if writer is not None:
                # Log loss and accuracy values using `writer`. Use `global_step` as a timestamp for the log.
                writer.add_scalar('test_loss', test_loss, global_step)
                writer.add_scalar('test_accuracy', test_accuracy, global_step)

                # Just for checking progress
                print(f'Test result of epoch {epoch}/{args.epoch} || loss : {test_loss:.3f} acc : {test_accuracy:.3f} ')

                writer.flush()

            # Whenever `test_accuracy` is greater than `best_accuracy`, save network weights with the filename 'best.pt' in the directory specified by `ckpt_dir`.
            if test_accuracy > best_accuracy:
                best_accuracy = test_accuracy
                torch.save(net.state_dict(), f'{ckpt_dir}/{block_type}_best.pt')

        scheduler.step()
    return best_accuracy


In [ ]:
# Function for weight initialization.
def weight_init(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        torch.nn.init.constant_(m.weight, 1)
        torch.nn.init.constant_(m.bias, 0)

In [ ]:
# List of all block types we will use.
block_types = ['mlp', 'conv','resPlain','resBottleneck','inception']

# Create directory name.
num_trial=0
parent_dir = result_dir / f'trial_{num_trial}'
while parent_dir.is_dir():
    num_trial = int(parent_dir.name.replace('trial_',''))
    parent_dir = result_dir / f'trial_{num_trial+1}'
print(f'Logs and ckpts will be saved in : {parent_dir}')

# Define networks
networks = []
for block_type in block_types:
    if block_type == 'conv':
        args.num_blocks = [10, 10, 10]
    else:
        args.num_blocks = [5, 5, 5]

    if block_type == 'mlp':
        network = MyNetworkExample(64, block_type).to(device)
    else:
        network = MyNetwork(args.num_filters, block_type, args.num_blocks).to(device)

    network.apply(weight_init)
    networks.append(network)

# Count the number of parameters of the models.
# You can use it as an indicator of whether you correctly implemented the model.

correct_params = {'mlp' : 1649354, 'conv' : 510426, 'resPlain' : 510426, 'resBottleneck' : 113946, 'inception' : 124026}
for block_type, net  in zip(block_types, networks):
    # Print the number of parameters in each model.
    num_parameters = sum(p.numel() for p in net.parameters() if p.requires_grad)
    print(f'# of parameters in {block_type} net : {num_parameters}')
    print(f'Correct # of parameters in {block_type} net : {correct_params[block_type]}')

In [ ]:
final_accs = {}

# Start training
for block_type, net in zip(block_types, networks):
    try:
        args.name = block_type
        # Define optimizer
        optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.0001)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,80], gamma=0.5)

        # Create directories for logs and ckechpoints.
        ckpt_dir = parent_dir / args.name / args.ckpt_dir
        ckpt_dir.mkdir(parents=True, exist_ok=True)
        log_dir = parent_dir / args.name / args.log_dir
        log_dir.mkdir(parents=True, exist_ok=True)

        # Create tensorboard writer,
        if args.tensorboard:
            writer = SummaryWriter(log_dir)

        # Call the train & test function.
        t1 = time.time()
        accuracy = train_net(net, optimizer, scheduler, block_type, writer)
        t = time.time()-t1
        print(f'Best test accuracy of {block_type} network : {accuracy:.3f} took {t:.3f} secs')
        final_accs[f'{block_type}'] = accuracy*100
    except Exception as e:
        print(e)

# Print final best accuracies of the models.
for key in final_accs.keys():
    print(f'Best accuracy of {key} = {final_accs[key]:.2f}%')


In [ ]:
block_types = ['mlp', 'conv','resPlain','resBottleneck','inception']
test_accs = {}
test_params= {}

for block_type, net in zip(block_types, networks):
        ckpt_dir = parent_dir / block_type / args.ckpt_dir

        # load weights from best checkpoints.
        ckpt_path = f'{ckpt_dir}/{block_type}_best.pt'
        try:
            net.load_state_dict(torch.load(ckpt_path))
        except Exception as e:
            print(e)

        # Measure test performance.
        net.eval()
        with torch.no_grad():
            test_accuracy = 0.
            test_num_data = 0.
            for batch_idx, (x, y) in enumerate(test_dataloader):
                # Send `x` and `y` to either cpu or gpu using `device` variable..
                x = x.to(device=device)
                y = y.to(device=device)

                # Feed `x` into the network, get an output, and keep it in a variable called `logit`.
                logit = net(x)

                # Compute loss using `logit` and `y`, and keep it in a variable called `loss`.
                loss = nn.CrossEntropyLoss()(logit, y)

                # Compute accuracy of this batch using `logit`, and keep it in a variable called 'accuracy'.
                accuracy = (logit.argmax(dim=1) == y).float().mean()

                test_accuracy += accuracy.item()*x.shape[0]
                test_num_data += x.shape[0]

            # Average classification accuracy.
            test_accuracy /= test_num_data

            # Count the number of implemented models.
            num_parameters = sum(p.numel() for p in net.parameters() if p.requires_grad)

            test_accs[f'{block_type}'] = test_accuracy*100
            test_params[f'{block_type}'] = num_parameters


In [ ]:
# Printing final results.
correct_accs = {'mlp' : 62.6,'conv' : 81.9,'resPlain' : 88.6, 'resBottleneck' : 86.5, 'inception' : 83.7}
correct_params = {'mlp' : 1649354, 'conv' : 510426, 'resPlain' : 510426, 'resBottleneck' : 113946, 'inception' : 124026}

print(' Method        | Accuracy   | # Params    | Expected Acc | Expected # Params  ')
print('------------------------------------------------------------------------------')
for block in block_types:
        print(f' {block:14}| {str(test_accs[block])[:5]:11}| {str(test_params[block]):11} | {str(correct_accs[block])[:5]:13}| {str(correct_params[block]):12}')